In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [2]:
import pandas as pd

df_features = pd.read_parquet("data/features_25m.parquet")
label_side = np.load("data/label_side_25m.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(106174, 11841)
(106174, 7)


2h_williams_r_ddt_lag39               808
2h_evenbetter_sinewave_short_lag28    808
2h_evenbetter_sinewave_short_lag20    808
2h_evenbetter_sinewave_short_lag21    808
2h_evenbetter_sinewave_short_lag22    808
                                     ... 
25m_comb_spectrum_pwr_9                 0
25m_comb_spectrum_pwr_8                 0
25m_comb_spectrum_pwr_7                 0
25m_comb_spectrum_pwr_6                 0
25m_ac_37                               0
Length: 11841, dtype: int64

In [3]:
df_features = df_features[808:]
label_side = label_side[808:]

print(df_features.shape)
print(label_side.shape)

label_side

(105366, 11841)
(105366, 7)


array([[ 1.58600034e+12,  6.82226000e+03,  6.70108000e+03, ...,
         6.62999000e+03,  3.30846160e+04, -1.00000000e+00],
       [ 1.58600388e+12,  6.70102000e+03,  6.73412000e+03, ...,
         6.65787000e+03,  3.11395250e+04,  1.00000000e+00],
       [ 1.58601396e+12,  6.73412000e+03,  6.76520000e+03, ...,
         6.70833000e+03,  3.04400920e+04,  1.00000000e+00],
       ...,
       [ 1.73827176e+12,  1.04961400e+05,  1.05061000e+05, ...,
         1.04864200e+05,  2.00223200e+03, -1.00000000e+00],
       [ 1.73827518e+12,  1.05060900e+05,  1.05209100e+05, ...,
         1.04997700e+05,  1.94644700e+03, -1.00000000e+00],
       [ 1.73827968e+12,  1.05209100e+05,  1.04676900e+05, ...,
         1.04628300e+05,  1.96454500e+03, -1.00000000e+00]])

In [4]:
from custom_indicators.config import SIDE_ALL

side_features = df_features[SIDE_ALL]
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(105366, 2368)
(array([0, 1]), array([50766, 54600]))
(array([0, 1]), array([10135, 10939]))


In [5]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-08 18:28:04,923] A new study created in memory with name: no-name-be067263-6c6f-49e8-9a86-d7f032ca327e
[I 2025-03-08 18:29:17,241] Trial 0 finished with value: 0.8612488422477196 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 33, 'max_depth': 40, 'min_gain_to_split': 0.2526514275323875, 'min_data_in_leaf': 63, 'lambda_l1': 24.875827136430892, 'lambda_l2': 67.20378396623155, 'num_boost_round': 600}. Best is trial 0 with value: 0.8612488422477196.


precision: 0.7807847176701939, recall: 0.7622268945973124, f1: 0.7713942085299288


[I 2025-03-08 18:29:31,522] Trial 1 finished with value: 0.867576735011615 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 166, 'max_depth': 45, 'min_gain_to_split': 0.19195340084362575, 'min_data_in_leaf': 259, 'lambda_l1': 27.312295042019244, 'lambda_l2': 26.412747172508546, 'num_boost_round': 124}. Best is trial 1 with value: 0.867576735011615.


precision: 0.7805547926393848, recall: 0.779413109059329, f1: 0.7799835330710823


[I 2025-03-08 18:30:07,503] Trial 2 finished with value: 0.8691634594010206 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 178, 'max_depth': 44, 'min_gain_to_split': 0.3453876651682137, 'min_data_in_leaf': 237, 'lambda_l1': 43.5705879879008, 'lambda_l2': 73.48561669299627, 'num_boost_round': 383}. Best is trial 2 with value: 0.8691634594010206.


...

precision: 0.7799854174261757, recall: 0.7823384221592468, f1: 0.7811601478709324


[I 2025-03-08 20:42:57,021] Trial 98 finished with value: 0.8655690278326422 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 206, 'max_depth': 45, 'min_gain_to_split': 0.3428795644322034, 'min_data_in_leaf': 81, 'lambda_l1': 37.24769503144298, 'lambda_l2': 81.99211526611784, 'num_boost_round': 424}. Best is trial 85 with value: 0.8693945926897028.


precision: 0.7801684673136788, recall: 0.7789560288874668, f1: 0.7795617766799323


[I 2025-03-08 20:44:51,218] Trial 99 finished with value: 0.8688101885177222 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 170, 'max_depth': 51, 'min_gain_to_split': 0.26706714715722557, 'min_data_in_leaf': 299, 'lambda_l1': 13.968674668710008, 'lambda_l2': 72.36362003248024, 'num_boost_round': 401}. Best is trial 85 with value: 0.8693945926897028.


precision: 0.7799108361386589, recall: 0.7836182466404608, f1: 0.7817601459188327


In [6]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 32,
 'max_depth': 50,
 'min_gain_to_split': 0.3224598396344323,
 'min_data_in_leaf': 80,
 'lambda_l1': 11.077546467647013,
 'lambda_l2': 85.8427512785022,
 'num_boost_round': 379}

In [7]:
params = {
    # "is_unbalance": True,
    **study.best_params,
}

dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

In [8]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
import plotly.express as px

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8688723712647338


In [10]:
model.save_model("custom_indicators/models/model_side.txt")

# meta model

In [12]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_45m.parquet")
df_label = pd.read_parquet("data/label_meta_45m.parquet")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
model_side_res = model_side.predict(df_features[SIDE_ALL])
df_features["model_side_res"] = model_side_res

In [13]:
df_label["ret"].notna().sum()

14896

In [14]:
df_features = df_features[583:]
df_label = df_label[583:]

print(df_features.shape)

df_features = df_features[df_label["ret"].notna()]
df_label = df_label[df_label["ret"].notna()]

assert df_features.shape[0] == df_label.shape[0]
print(df_label.shape)
df_label.head()

(58554, 11842)
(14736, 9)


,open,high,low,close,volume,ret,trgt,bin,side
index,,,,,,,,,
2020-04-28 21:54:00+00:00,7716.02,7879.99,7712.92,7842.79,48845.750,0.034708,0.010402,1.0,1.0
2020-04-29 02:08:00+00:00,7843.44,7843.44,7710.00,7820.51,48915.848,0.037656,0.010812,1.0,1.0
2020-04-29 04:45:00+00:00,7820.39,7955.00,7819.59,7941.16,48157.749,0.021891,0.010790,1.0,1.0
2020-04-29 06:35:00+00:00,7941.16,7965.99,7909.54,7958.77,49086.039,0.022067,0.009961,1.0,1.0
2020-04-29 09:43:00+00:00,7958.77,8128.00,7940.22,8115.00,47613.448,0.030865,0.010091,1.0,1.0


In [15]:
from custom_indicators.config import META_ALL

meta_features = df_features[META_ALL]
print(meta_features.shape)
meta_label = df_label["bin"].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features.iloc[:train_test_split_point]
meta_features_test = meta_features.iloc[train_test_split_point:]
meta_label_train = meta_label.iloc[:train_test_split_point]
meta_label_test = meta_label.iloc[train_test_split_point:]

meta_label_test.value_counts()

(14736, 4438)


bin
1    1526
0    1422
Name: count, dtype: int64

In [16]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        # "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    auc = roc_auc_score(meta_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}")
    return f1


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-03-07 12:46:14,445] A new study created in memory with name: no-name-77ea8ae0-9bd8-438a-ba1f-bcad2c0c7423
[I 2025-03-07 12:46:32,088] Trial 0 finished with value: 0.4182692307692308 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 287, 'max_depth': 42, 'min_gain_to_split': 0.5277602343585173, 'min_data_in_leaf': 106, 'lambda_l1': 92.56704217506919, 'lambda_l2': 28.4784572009264, 'num_boost_round': 900}. Best is trial 0 with value: 0.4182692307692308.


precision: 0.5381443298969072, recall: 0.3420707732634338, f1: 0.4182692307692308, auc: 0.493525492494834


[I 2025-03-07 12:46:36,865] Trial 1 finished with value: 0.3988786543852623 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 185, 'max_depth': 49, 'min_gain_to_split': 0.6293765158671873, 'min_data_in_leaf': 167, 'lambda_l1': 14.642195916689658, 'lambda_l2': 27.92793448097844, 'num_boost_round': 161}. Best is trial 0 with value: 0.4182692307692308.


precision: 0.5128733264675592, recall: 0.32634338138925295, f1: 0.3988786543852623, auc: 0.5022184618050372


[I 2025-03-07 12:46:44,121] Trial 2 finished with value: 0.4109697933227345 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 268, 'max_depth': 34, 'min_gain_to_split': 0.8162836423499501, 'min_data_in_leaf': 299, 'lambda_l1': 59.59317338076894, 'lambda_l2': 41.91567884385067, 'num_boost_round': 330}. Best is trial 0 with value: 0.4182692307692308.


...

precision: 0.5189075630252101, recall: 0.3237221494102228, f1: 0.3987086359967716, auc: 0.4971688574783453


[I 2025-03-07 13:09:04,265] Trial 98 finished with value: 0.4082960553070354 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 227, 'max_depth': 48, 'min_gain_to_split': 0.4406399047694806, 'min_data_in_leaf': 50, 'lambda_l1': 99.7798375814652, 'lambda_l2': 21.752289546133284, 'num_boost_round': 241}. Best is trial 66 with value: 0.4438291139240506.


precision: 0.5380493033226152, recall: 0.3289646133682831, f1: 0.4082960553070354, auc: 0.5029320654828726


[I 2025-03-07 13:09:09,213] Trial 99 finished with value: 0.39476910502656315 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 202, 'max_depth': 41, 'min_gain_to_split': 0.5839121615716808, 'min_data_in_leaf': 39, 'lambda_l1': 88.47473446149459, 'lambda_l2': 0.5452426642753405, 'num_boost_round': 211}. Best is trial 66 with value: 0.4438291139240506.


precision: 0.5244299674267101, recall: 0.3165137614678899, f1: 0.39476910502656315, auc: 0.497758726840715


In [17]:
study.best_params

{'extra_trees': True,
 'boosting': 'gbdt',
 'num_leaves': 274,
 'max_depth': 57,
 'min_gain_to_split': 0.6606789380825927,
 'min_data_in_leaf': 101,
 'lambda_l1': 96.27579805878213,
 'lambda_l2': 14.860350102315499,
 'num_boost_round': 1000}

In [18]:
params = {
    "is_unbalance": True,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

In [20]:
import plotly.express as px

res = model.predict(meta_features_test)

fig = px.histogram(res, nbins=100)
fig.show()

In [30]:
import plotly.express as px

df_features = pd.read_parquet("data/features_45m.parquet")
model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
model_side_res = model_side.predict(df_features[SIDE_ALL])
df_features["model_side_res"] = model_side_res
df_features = df_features[META_ALL]
df_label = pd.read_parquet("data/label_meta_45m.parquet")

extra_features_test = df_features[int(len(df_features) * 0.8) :]
extra_label = df_label[int(len(df_features) * 0.8) :]

extra_features_test = extra_features_test[df_label["ret"].isna()]
extra_label = extra_label[df_label["ret"].isna()]

assert len(extra_features_test) == len(extra_label)

res = model.predict(extra_features_test)

fig = px.histogram(res, nbins=100)
fig.show()

/var/folders/h9/d_xpszfn6yz9r8zww450m44w0000gn/T/ipykernel_30270/2819802013.py:13: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/h9/d_xpszfn6yz9r8zww450m44w0000gn/T/ipykernel_30270/2819802013.py:14: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [21]:
model.save_model("custom_indicators/models/model_meta.txt")